# 神经网络训练整体过程

> 将所有的原始图片信息存储为TFRecord格式

> 通过文件列表创建输入文件队列

> 解析TFRecord中的像素矩阵，并根据尺寸还原原始图像

> 图像预处理----裁剪+变换等

> 将处理后的图像和标签数据整理成神经网络输入需要的batch

> 定义神经网络结构及优化过程

> 声明会话并运行神经网络的优化过程

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 随机调整一张图片的色彩
# 因为调整亮度，对比度，饱和度，色相的顺序会影响最后得到的结果，所以可以定义多长不同的顺序
# 可以在训练数据预处理时随机选择一种，这样可以进一步降低无关因素对模型的影响
def distort_color(image, color_ordering=0):
    if color_ordering == 0:
        image = tf.image.random_brightness(image, max_delta=32./255.)
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        image = tf.image.random_hue(image, max_delta=0.2)
        image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    else:
        image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
        image = tf.image.random_brightness(image, max_delta=32./255.)
        image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
        image = tf.image.random_hue(image, max_delta=0.2)
    return tf.clip_by_value(image, 0.0, 1.0)

# 对图片进行预处理，转化成神经网络的输入层数据
# 输入参数：一张解码后的图像、目标图像的尺寸以及图像上的标注框
def preprocess_for_train(image, height, width, bbox):
    # 如果没有提供标注框，则认为整个图像就是需要关注的部分
    if bbox is None:
        bbox = tf.constant([0.0, 0.0, 1.0, 1.0], dtype=tf.float32, shape=[1, 1, 4])
        
    # 转换图像张量的类型
    if image.dtype != tf.float32:
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        
    # 随机截取图像，减小需要关注的物体大小对图像识别算法的影响
    bbox_begin, bbox_size, _ = tf.image.sample_distorted_bounding_box(tf.shape(image), bounding_boxes=bbox)
    print "bbox_bengin: ", bbox_begin, " ,bbox_size: ", bbox_size
    distorted_image = tf.slice(image, bbox_begin, bbox_size)
    
    # 将随机截取的图像调整为神经网络的输入层的大小。大小调整的算法是随机的
    distorted_image = tf.image.resize_images(distorted_image, height, width, method=np.random.randint(4))
    
    # 随机左右翻转图像
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    
    # 使用一种随机的顺序调整图像色彩
    distorted_image = distort_color(distorted_image, np.random.randint(2))
    
    return distorted_image


# 通过文件列表创建输入文件队列
# 在调用输入数据处理流程前，需要统一所有原始数据的格式并存储到TFRecord文件中
files = tf.train.match_filenames_once("/Users/xxx/tmp/data.tfrecords-*")
filename_queue = tf.train.string_input_producer(files, shuffle=False)


# 解析TFRecord文件里的数据
# image存储的是图像的原始数据
# label为样例所对应的标签
# height，width，channels给出来图片的维度
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'image': tf.FixedLenFeature([], tf.string),
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'height': tf.FixedLenFeature([], tf.int64),
                                       'width': tf.FixedLenFeature([], tf.int64),
                                       'channels': tf.FixedLenFeature([], tf.int64)
                                   })

image, label = features['image'], features['label']
height, width = features['height'], features['width']
channels = features['channels']

# 从原始图像数据解析出像素矩阵，并根据图像尺寸还原图像
decoded_image = tf.decode_raw(image, tf.uint8)
decoded_image.set_shape([height, width, channels])

# 定义神经网络输入层图片的大小
image_size = 299
# 图像预处理
distorted_image = preprocess_for_train(decoded_image, image_size, image_size, None)

# 将处理后的图像和标签数据整理成神经网络训练需要的batch
min_after_dequeue = 10000
batch_size = 100
capacity = min_after_dequeue + 3 * batch_size
image_batch, label_batch = tf.train.shuffle_batch([distorted_image, label], batch_size=batch_size,
                                                 capacity=capacity, min_after_dequeue=min_after_dequeue)

# 定义神经网络的结构及优化过程
logit = inference(image_batch)
loss = calc_loss(logit, label_batch)
train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

# 声明会话并运行神经网络的优化过程
with tf.Session() as sess:
    # 神经网络的准备工作--变量初始化、线程启动
    tf.initialize_all_variables().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 神经网络训练过程
    for i in range(TRAINNING_ROUNDS):
        sess.run(train_step)
    
    
    # 停止所有线程
    coord.request_stop()
    coord.join(threads)

